# Deep Ranking Model End-to-End Demo

This notebook demonstrates KMR's DeepRankingModel for learning-to-rank recommendations, including:

- Data generation using KMR utilities
- Model creation and training with recommendation metrics
- Recommendation generation and evaluation
- Visualization of recommendations


In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras.optimizers import Adam
from keras.losses import BinaryCrossentropy

from kmr.models import DeepRankingModel
from kmr.metrics import AccuracyAtK, PrecisionAtK, RecallAtK
from kmr.utils import KMRDataGenerator, KMRPlotter

print("✅ All imports successful!")
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")


✅ All imports successful!
TensorFlow version: 2.18.0
Keras version: 3.8.0


## 1. Generate Content-Based Recommendation Data

We'll use KMR's data generator to create synthetic user and item features with interactions.


In [2]:
print("📦 Generating content-based recommendation data...")

user_features, item_features, user_ids, item_ids, ratings = KMRDataGenerator.generate_content_based_recommendation_data(
    n_users=1000,
    n_items=500,
    user_feature_dim=20,
    item_feature_dim=15,
    n_interactions=10000,
    random_state=42
)

n_users = len(np.unique(user_ids))
n_items = len(np.unique(item_ids))

print(f"✅ Generated data:")
print(f"   - Users: {n_users}")
print(f"   - Items: {n_items}")
print(f"   - User features: {user_features.shape}")
print(f"   - Item features: {item_features.shape}")
print(f"   - Interactions: {len(user_ids)}")
print(f"   - Rating range: {ratings.min():.1f} - {ratings.max():.1f}")

# Convert to binary interaction (for implicit feedback)
interactions = (ratings >= 3.0).astype(np.float32)

# Split into train/test
train_size = int(0.8 * len(user_ids))
train_user_ids = user_ids[:train_size]
train_item_ids = item_ids[:train_size]
train_interactions = interactions[:train_size]

test_user_ids = user_ids[train_size:]
test_item_ids = item_ids[train_size:]
test_interactions = interactions[train_size:]


📦 Generating content-based recommendation data...
✅ Generated data:
   - Users: 1000
   - Items: 500
   - User features: (1000, 20)
   - Item features: (500, 15)
   - Interactions: 10000
   - Rating range: 1.0 - 5.0


## 2. Build Deep Ranking Model


In [3]:
# Create model
model = DeepRankingModel(
    user_feature_dim=20,
    item_feature_dim=15,
    num_items=n_items,
    hidden_units=[128, 64, 32],
    activation="relu",
    dropout_rate=0.3,
    batch_norm=True,
    l2_reg=1e-4,
    top_k=10
)

# Create recommendation metrics
acc_at_5 = AccuracyAtK(k=5, name="acc@5")
acc_at_10 = AccuracyAtK(k=10, name="acc@10")
prec_at_5 = PrecisionAtK(k=5, name="prec@5")
prec_at_10 = PrecisionAtK(k=10, name="prec@10")
recall_at_5 = RecallAtK(k=5, name="recall@5")
recall_at_10 = RecallAtK(k=10, name="recall@10")

# Compile model with recommendation metrics
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss=BinaryCrossentropy(),
    metrics=[acc_at_5, acc_at_10, prec_at_5, prec_at_10, recall_at_5, recall_at_10]
)

print("✅ Model created and compiled!")
print(f"   - User feature dim: {model.user_feature_dim}")
print(f"   - Item feature dim: {model.item_feature_dim}")
print(f"   - Hidden units: {model.hidden_units}")
print(f"   - Top-K: {model.top_k}")
print(f"   - Metrics: Accuracy@5, Accuracy@10, Precision@5, Precision@10, Recall@5, Recall@10")


2025-11-05 17:42:26.230 | DEBUG    | kmr.layers._base_layer:_log_initialization:73 - Initialized DeepFeatureRanking with parameters: {'name': 'deep_feature_ranking', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'hidden_dim': 128, 'l2_reg': 0.0001, 'dropout_rate': 0.3}
2025-11-05 17:42:26.241 | DEBUG    | kmr.layers._base_layer:_log_initialization:73 - Initialized TopKRecommendationSelector with parameters: {'name': 'top_k_recommendation_selector', 'trainable': True, 'dtype': {'module': 'keras', 'class_name': 'DTypePolicy', 'config': {'name': 'float32'}, 'registered_name': None}, 'k': 10}
2025-11-05 17:42:26.242 | DEBUG    | kmr.models.DeepRankingModel:__init__:138 - Initialized deep_ranking_model with user_dim=20, item_dim=15, top_k=10
2025-11-05 17:42:26.244 | DEBUG    | kmr.metrics.accuracy_at_k:__init__:71 - Initialized AccuracyAtK metric with k=5, name=acc@5
2025-11-05 17:42:26.244 | DEBUG    |

✅ Model created and compiled!
   - User feature dim: 20
   - Item feature dim: 15
   - Hidden units: [128, 64, 32]
   - Top-K: 10
   - Metrics: Accuracy@5, Accuracy@10, Precision@5, Precision@10, Recall@5, Recall@10


/Users/piotrlaczkowski/Library/Caches/pypoetry/virtualenvs/kmr-S1SSCx8j-py3.12/lib/python3.12/site-packages/keras/src/backend/tensorflow/trainer.py:642: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


## 3. Train Model


In [4]:
print("🚀 Training Model")
print("=" * 60)
print("Using model.fit() with built-in ranking loss")
print("=" * 60)
print("The model's train_step() method handles ranking loss internally!")
print("Just prepare data and call model.fit() - no custom training loop needed.\n")

# Prepare data for keras.fit() format
# For each user, provide all items and binary labels
unique_users = np.unique(train_user_ids)[:50]  # Use subset for demo
# Filter to only valid user IDs (within range of user_features)
unique_users = unique_users[unique_users < len(user_features)]
batch_size = 8

# Create training data: for each user, provide all items and binary labels
train_x_user_features = []
train_x_item_features = []
train_y = []

for user_id in unique_users:
    # Get user's features (user_id directly indexes into user_features)
    user_feat = user_features[user_id]
    
    # Get user's positive items
    user_item_ids = train_item_ids[train_user_ids == user_id]
    positive_set = set(user_item_ids[user_item_ids < n_items])  # Filter valid items
    
    # Create label vector: 1 for positive items, 0 for others
    labels = np.zeros(n_items, dtype=np.float32)
    labels[list(positive_set)] = 1.0
    
    # Prepare item features: all items for this user
    item_feats = item_features[:n_items]  # (n_items, item_feature_dim)
    
    train_x_user_features.append(user_feat)
    train_x_item_features.append(item_feats)
    train_y.append(labels)

train_x_user_features = np.array(train_x_user_features, dtype=np.float32)
train_x_item_features = np.array(train_x_item_features, dtype=np.float32)
train_y = np.array(train_y, dtype=np.float32)

print(f"Prepared training data: {len(train_x_user_features)} users")
print(f"  - User features shape: {train_x_user_features.shape}")
print(f"  - Item features shape: {train_x_item_features.shape}")
print(f"  - Labels shape: {train_y.shape}")
print(f"  - Positive items per user: {train_y.sum(axis=1).mean():.1f} on average\n")

# Build model by calling it once with sample data
# This ensures all layers are initialized before training
_ = model([tf.constant(train_x_user_features[:1]), tf.constant(train_x_item_features[:1])], training=False)

print("Training with model.fit()...")
history = model.fit(
    x=[train_x_user_features, train_x_item_features],
    y=train_y,
    epochs=3,
    batch_size=batch_size,
    verbose=1
)

print("\n✅ Training completed!")
print(f"Final loss: {history.history['loss'][-1]:.4f}")

# Display recommendation metrics
if 'acc@5' in history.history:
    print("\n📊 Recommendation Metrics:")
    print(f"   - Accuracy@5:  {history.history['acc@5'][-1]:.4f}")
    print(f"   - Accuracy@10: {history.history['acc@10'][-1]:.4f}")
    print(f"   - Precision@5:  {history.history['prec@5'][-1]:.4f}")
    print(f"   - Precision@10: {history.history['prec@10'][-1]:.4f}")
    print(f"   - Recall@5:  {history.history['recall@5'][-1]:.4f}")
    print(f"   - Recall@10: {history.history['recall@10'][-1]:.4f}")

print("\nNote: The model uses margin ranking loss internally.")
print("      Positive items are encouraged to rank higher than negative items.")
print("      Metrics track recommendation quality: Accuracy@K, Precision@K, Recall@K.")


🚀 Training Model
Using model.fit() with built-in ranking loss
The model's train_step() method handles ranking loss internally!
Just prepare data and call model.fit() - no custom training loop needed.

Prepared training data: 50 users
  - User features shape: (50, 20)
  - Item features shape: (50, 500, 15)
  - Labels shape: (50, 500)
  - Positive items per user: 7.5 on average

Training with model.fit()...
Epoch 1/3
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - acc@10: 0.0000e+00 - acc@5: 0.0000e+00 - loss: 0.8799 - prec@10: 0.0000e+00 - prec@5: 0.0000e+00 - recall@10: 0.0000e+00 - recall@5: 0.0000e+00  
Epoch 2/3
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - acc@10: 0.0000e+00 - acc@5: 0.0000e+00 - loss: 0.8868 - prec@10: 0.0000e+00 - prec@5: 0.0000e+00 - recall@10: 0.0000e+00 - recall@5: 0.0000e+00 
Epoch 3/3
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - acc@10: 0.0000e+00 - acc@5: 0.0000e+00 - loss: 0.8857 - prec@10: 0.0000e+00 - prec@5: 0.0000e+00 - recall@10: 0.0000e+00 - recall@5: 0.0000e+00 

✅ Train

## 4. Generate Recommendations and Visualize


In [5]:
# Generate recommendations for a sample user
sample_user_idx = 0
sample_user_feat = tf.constant([user_features[sample_user_idx]])
sample_item_feats = tf.constant([item_features[:n_items]])

rec_indices, rec_scores = model([sample_user_feat, sample_item_feats], training=False)

print(f"✅ Generated recommendations for user {sample_user_idx}")
print(f"   Top-{model.top_k} recommended items: {rec_indices[0].numpy()}")
print(f"   Recommendation scores: {rec_scores[0].numpy()}")

# Visualize recommendation scores
fig = KMRPlotter.plot_recommendation_scores(
    rec_scores[0].numpy(),
    top_k=model.top_k,
    title="Recommendation Scores for Sample User"
)
fig.show()


✅ Generated recommendations for user 0
   Top-10 recommended items: [166 498  98 490 444 486 432 159 314 126]
   Recommendation scores: [0.52085197 0.51720923 0.5162965  0.5154507  0.5148066  0.513605
 0.51347846 0.5128926  0.5108594  0.5083008 ]
